Learn how to summarize the columns available in an R data frame. 
  You will also learn how to chain operations together with the
  pipe operator, and how to compute grouped summaries using.

## Welcome!

Hey there! Ready for the first lesson?

The dfply package makes it possible to do R's dplyr-style data manipulation with pipes in python on pandas DataFrames.

[dfply website here](https://github.com/kieferk/dfply)

[![](https://www.rforecology.com/pipes_image0.png "https://github.com/kieferk/dfply"){width="600"}](https://github.com/kieferk/dfply)

In [5]:
import pandas as pd
import seaborn as sns
from pandas import Series

cars = sns.load_dataset('mpg')
from dfply import *
cars >> head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite


## The \>\> and \>\>=

dfply works directly on pandas DataFrames, chaining operations on the data with the >> operator, or alternatively starting with >>= for inplace operations.

*The X DataFrame symbol*

The DataFrame as it is passed through the piping operations is represented by the symbol X. It records the actions you want to take (represented by the Intention class), but does not evaluate them until the appropriate time. Operations on the DataFrame are deferred. Selecting two of the columns, for example, can be done using the symbolic X DataFrame during the piping operations.

### Exercise 1.

Select the columns 'mpg' and 'horsepower' from the cars DataFrame.

In [6]:
cars >> select(X.mpg, X.horsepower) >> head(3)

,mpg,horsepower
0,18.0,130.0
1,15.0,165.0
2,18.0,150.0


## Selecting and dropping

There are two functions for selection, inverse of each other: select and drop. The select and drop functions accept string labels, integer positions, and/or symbolically represented column names (X.column). They also accept symbolic "selection filter" functions, which will be covered shortly.

### Exercise 2.

Select the columns 'mpg' and 'horsepower' from the cars DataFrame using the drop function.

In [7]:
cars >> drop(X.weight, X.origin, X.cylinders, X.displacement, X.acceleration,  X.model_year, X.name) >> head(3)

,mpg,horsepower
0,18.0,130.0
1,15.0,165.0
2,18.0,150.0


## Selection using \~

One particularly nice thing about dplyr's selection functions is that you can drop columns inside of a select statement by putting a subtraction sign in front, like so: ... %>% select(-col). The same can be done in dfply, but instead of the subtraction operator you use the tilde ~.

### Exercise 3.

Select all columns except 'model_year', and 'name' from the cars DataFrame.

In [8]:
cars >> select(~X.model_year, ~X.name) >> head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,origin
0,18.0,8,307.0,130.0,3504,12.0,usa
1,15.0,8,350.0,165.0,3693,11.5,usa
2,18.0,8,318.0,150.0,3436,11.0,usa


## Filtering columns

The vanilla select and drop functions are useful, but there are a variety of selection functions inspired by dplyr available to make selecting and dropping columns a breeze. These functions are intended to be put inside of the select and drop functions, and can be paired with the ~ inverter.

First, a quick rundown of the available functions:

-   starts_with(prefix): find columns that start with a string prefix.
-   ends_with(suffix): find columns that end with a string suffix.
-   contains(substr): find columns that contain a substring in their name.
-   everything(): all columns.
-   columns_between(start_col, end_col, inclusive=True): find columns between a specified start and end column. The inclusive boolean keyword argument indicates whether the end column should be included or not.
-   columns_to(end_col, inclusive=True): get columns up to a specified end column. The inclusive argument indicates whether the ending column should be included or not.
-   columns_from(start_col): get the columns starting at a specified column.

### Exercise 4.

The selection filter functions are best explained by example. Let's say I wanted to select only the columns that started with a "c":

In [9]:
cars >> select(starts_with('c')) >> head(3)

,cylinders
0,8
1,8
2,8


### Exercise 5.

Select the columns that contain the substring "e" from the cars DataFrame.

In [10]:
cars >> select(contains('e')) >> head(3)

,cylinders,displacement,horsepower,weight,acceleration,model_year,name
0,8,307.0,130.0,3504,12.0,70,chevrolet chevelle malibu
1,8,350.0,165.0,3693,11.5,70,buick skylark 320
2,8,318.0,150.0,3436,11.0,70,plymouth satellite


### Exercise 6.

Select the columns that are between 'mpg' and 'origin' from the cars DataFrame.

In [11]:
cars >> select(columns_between('mpg', 'origin')) >> head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,usa
1,15.0,8,350.0,165.0,3693,11.5,70,usa
2,18.0,8,318.0,150.0,3436,11.0,70,usa


## Subsetting and filtering

### row_slice()

Slices of rows can be selected with the row_slice() function. You can pass single integer indices or a list of indices to select rows as with. This is going to be the same as using pandas' .iloc.

#### Exercise 7.

Select the first three rows from the cars DataFrame.

In [12]:
cars >> row_slice([0, 1, 2])

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite


### distinct()

Selection of unique rows is done with distinct(), which similarly passes arguments and keyword arguments through to the DataFrame's .drop_duplicates() method.

#### Exercise 8.

Select the unique rows from the 'origin' column in the cars DataFrame.

In [13]:
cars >> distinct(X.origin)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
14,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii
19,26.0,4,97.0,46.0,1835,20.5,70,europe,volkswagen 1131 deluxe sedan


## mask()

Filtering rows with logical criteria is done with mask(), which accepts boolean arrays "masking out" False labeled rows and keeping True labeled rows. These are best created with logical statements on symbolic Series objects as shown below. Multiple criteria can be supplied as arguments and their intersection will be used as the mask.

### Exercise 9.

Filter the cars DataFrame to only include rows where the 'mpg' is greater than 20, origin Japan, and display the first three rows:

In [17]:
cars >> mask(X.mpg > 20, X.origin == 'japan') >> head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
14,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii
18,27.0,4,97.0,88.0,2130,14.5,70,japan,datsun pl510
29,27.0,4,97.0,88.0,2130,14.5,71,japan,datsun pl510


## pull()

The pull() function is used to extract a single column from a DataFrame as a pandas Series. This is useful for passing a single column to a function or for further manipulation.

### Exercise 10.

Extract the 'mpg' column from the cars DataFrame, japanese origin, model year 70s, and display the first three rows.

In [19]:
cars >> mask(X.model_year == 70, X.origin == 'japan') >> head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
14,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii
18,27.0,4,97.0,88.0,2130,14.5,70,japan,datsun pl510


## DataFrame transformation

*mutate()*

The mutate() function is used to create new columns or modify existing columns. It accepts keyword arguments of the form new_column_name = new_column_value, where new_column_value is a symbolic Series object.

### Exercise 11.

Create a new column 'mpg_per_cylinder' in the cars DataFrame that is the result of dividing the 'mpg' column by the 'cylinders' column.

In [20]:
cars >> transmute(mpg_per_cylinder=X.mpg / X.cylinders)  >> head(3)

,mpg_per_cylinder
0,2.250
1,1.875
2,2.250



*transmute()*

The transmute() function is a combination of a mutate and a selection of the created variables.

### Exercise 12.

Create a new column 'mpg_per_cylinder' in the cars DataFrame that is the result of dividing the 'mpg' column by the 'cylinders' column, and display only the new column.

In [21]:
cars >> transmute(mpg_per_cylinder=X.mpg / X.cylinders) >> select(X.mpg_per_cylinder) >> head(3)

,mpg_per_cylinder
0,2.250
1,1.875
2,2.250


## Grouping

*group_by() and ungroup()*

The group_by() function is used to group the DataFrame by one or more columns. This is useful for creating groups of rows that can be summarized or transformed together. The ungroup() function is used to remove the grouping.

### Exercise 13.

Group the cars DataFrame by the 'origin' column and calculate the lead of the 'mpg' column.

In [22]:
cars >> group_by(X.origin) >> mutate(mpg_lead=lead(X.mpg)) >> head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name,mpg_lead
19,26.0,4,97.0,46.0,1835,20.5,70,europe,volkswagen 1131 deluxe sedan,25.0
20,25.0,4,110.0,87.0,2672,17.5,70,europe,peugeot 504,24.0
21,24.0,4,107.0,90.0,2430,14.5,70,europe,audi 100 ls,25.0
14,24.0,4,113.0,95.0,2372,15.0,70,japan,toyota corona mark ii,27.0
18,27.0,4,97.0,88.0,2130,14.5,70,japan,datsun pl510,27.0
29,27.0,4,97.0,88.0,2130,14.5,71,japan,datsun pl510,25.0
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu,15.0
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320,18.0
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite,16.0


## Reshaping

*arrange()*

The arrange() function is used to sort the DataFrame by one or more columns. This is useful for reordering the rows of the DataFrame.

### Exercise 14.

Sort the cars DataFrame by the 'mpg' column in descending order.

In [24]:
cars >> arrange(X.mpg, ascending=True) >> head(3)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
28,9.0,8,304.0,193.0,4732,18.5,70,usa,hi 1200d
25,10.0,8,360.0,215.0,4615,14.0,70,usa,ford f250
26,10.0,8,307.0,200.0,4376,15.0,70,usa,chevy c20



*rename()*

The rename() function is used to rename columns in the DataFrame. It accepts keyword arguments of the form new_column_name = old_column_name.

### Exercise 15.

Rename the 'mpg' column to 'miles_per_gallon' in the cars DataFrame.

In [26]:
cars >> rename(miles_per_gallon=X.mpg) >> head(3)

,miles_per_gallon,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite



*gather()*

The gather() function is used to reshape the DataFrame from wide to long format. It accepts keyword arguments of the form new_column_name = new_column_value, where new_column_value is a symbolic Series object.

### Exercise 16.

Reshape the cars DataFrame from wide to long format by gathering the columns 'mpg', 'horsepower', 'weight', 'acceleration', and 'displacement' into a new column 'variable' and their values into a new column 'value'.

In [42]:
cars_long = cars >> gather('variable', 'value', columns_from=['mpg', 'horsepower', 'weight', 'acceleration', 'displacement'])

variable    object
value       object
dtype: object


*spread()*

Likewise, you can transform a "long" DataFrame into a "wide" format with the spread(key, values) function. Converting the previously created elongated DataFrame for example would be done like so.

### Exercise 17.

Reshape the cars DataFrame from long to wide format by spreading the 'variable' column into columns and their values into the 'value' column.

In [61]:
cars_long['ID'] = cars_long.groupby('variable').cumcount()
cars_long >> spread(X.variable, X.value) >> head(20)

,ID,acceleration,cylinders,displacement,horsepower,model_year,mpg,name,origin,weight
0,0,12.0,8,307.0,130.0,70,18.0,chevrolet chevelle malibu,usa,3504
1,1,11.5,8,350.0,165.0,70,15.0,buick skylark 320,usa,3693
2,2,11.0,8,318.0,150.0,70,18.0,plymouth satellite,usa,3436
3,3,12.0,8,304.0,150.0,70,16.0,amc rebel sst,usa,3433
4,4,10.5,8,302.0,140.0,70,17.0,ford torino,usa,3449
5,5,10.0,8,429.0,198.0,70,15.0,ford galaxie 500,usa,4341
6,6,9.0,8,454.0,220.0,70,14.0,chevrolet impala,usa,4354
7,7,8.5,8,440.0,215.0,70,14.0,plymouth fury iii,usa,4312
8,8,10.0,8,455.0,225.0,70,14.0,pontiac catalina,usa,4425
9,9,8.5,8,390.0,190.0,70,15.0,amc ambassador dpl,usa,3850



## Summarization

*summarize()*

The summarize() function is used to calculate summary statistics for groups of rows. It accepts keyword arguments of the form new_column_name = new_column_value, where new_column_value is a symbolic Series object.

### Exercise 18.

Calculate the mean 'mpg' for each group of 'origin' in the cars DataFrame.

In [46]:
cars >> group_by(X.origin) >> summarise(value=mean(X.mpg)) >> head(3)

,origin,value
0,europe,27.891429
1,japan,30.450633
2,usa,20.083534



*summarize_each()*

The summarize_each() function is used to calculate summary statistics for groups of rows. It accepts keyword arguments of the form new_column_name = new_column_value, where new_column_value is a symbolic Series object.

### Exercise 19.

Calculate the mean 'mpg' and 'horsepower' for each group of 'origin' in the cars DataFrame.

In [49]:
cars >> group_by(X.origin) >> summarize(mpg_mean=mean(X.mpg), hp_mean=mean(X.horsepower)) >> head(3)

,origin,mpg_mean,hp_mean
0,europe,27.891429,80.558824
1,japan,30.450633,79.835443
2,usa,20.083534,119.048980



*summarize() can of course be used with groupings as well.*

### Exercise 20.

Calculate the mean 'mpg' for each group of 'origin' and 'model_year' in the cars DataFrame.

In [50]:
cars >> group_by(X.origin, X.model_year) >> summarize(mpg_mean=mean(X.mpg)) >> head(3)

,model_year,origin,mpg_mean
0,70,europe,25.200000
1,71,europe,28.750000
2,72,europe,22.000000
3,73,europe,24.000000
4,74,europe,27.000000
5,75,europe,24.500000
6,76,europe,24.250000
7,77,europe,29.250000
8,78,europe,24.950000
9,79,europe,30.450000
